# NLP Exercise 4: Retrieval Augmented Generation
---


In [37]:
import os
import shutil
from langchain.document_loaders.pdf import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema.document import Document
from langchain.vectorstores.chroma import Chroma
from vector_embedding import embedding_function

In [36]:
# Define chroma and data path
chroma_path = 'Week_4\chroma'
data_path = 'Week_4\data'

In [38]:
# Load documents
def load_documents():
    document_loader = PyPDFDirectoryLoader(data_path)
    return document_loader.load()

documents = load_documents()

In [28]:
# Chunk documents
def split_documents(documents: list[Document]):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,
        chunk_overlap=80,
        length_function=len,
        is_separator_regex=False
    )
    return splitter.split_documents(documents)

chunks = split_documents(documents)

In [39]:
# Get chunk ids
def calculate_chunk_ids(chunks):

    # This will create IDs like "data/monopoly.pdf:6:2"
    # Page Source : Page Number : Chunk Index

    last_page_id = None
    current_chunk_index = 0

    for chunk in chunks:
        source = chunk.metadata.get("source")
        page = chunk.metadata.get("page")
        current_page_id = f"{source}:{page}"

        # If the page ID is the same as the last one, increment the index.
        if current_page_id == last_page_id:
            current_chunk_index += 1
        else:
            current_chunk_index = 0

        # Calculate the chunk ID.
        chunk_id = f"{current_page_id}:{current_chunk_index}"
        last_page_id = current_page_id

        # Add it to the page meta-data.
        chunk.metadata["id"] = chunk_id

    return chunks

In [40]:
def add_to_chroma(chunks: list[Document]):
    # Load the database
    db = Chroma(
        persist_directory=chroma_path, embedding_function=embedding_function()
    )

    chunks_ids = calculate_chunk_ids(chunks)

    # Add or update the documents
    existing_items = db.get(include=[])
    existing_ids = set(existing_items["ids"])
    print(f"Number of existing documents in DB: {len(existing_ids)}")

    new_chunks = []
    for chunk in chunks_ids:
        if chunk.metadata["id"] not in existing_ids:
            new_chunks.append(chunk)

    if len(new_chunks):
        print(f"Adding new documents: {len(new_chunks)}")
        new_chunks_ids = [chunk.metadata["id"] for chunk in new_chunks]
        db.add_documents(new_chunks, ids=new_chunks_ids)
        db.persist()
    else: 
        print("No new documents to add")

In [42]:
add_to_chroma(chunks)

Number of existing documents in DB: 47
No new documents to add
